In [ ]:
# Install required packages
!pip install -qU langchain==0.3.12 langchain-chroma langchain-community pypdf langchain-openai wikipedia langgraph

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.9/326.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Environment setup
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Eleven Labs

In [ ]:
# # from langchain_core.tools import tool

# # tool
# # def token_calculate(text: str) -> dict:
# #     """
# #     Calculates the number of characters in the input text and estimates Eleven Labs credit usage.
# #     Returns the character count and estimated cost in USD.
# #     """
# #     num_characters = len(text)
# #     cost_per_character = 0.0001  # $1 per 10,000 characters
# #     estimated_cost = round(num_characters * cost_per_character, 4)

# #     return {
# #         "character_count": num_characters,
# #         "estimated_cost_usd": estimated_cost
# #     }
# from langchain_core.tools import tool

# tool
# def token_calculate(text: str) -> dict:
#     """
#     Calculates Eleven Labs TTS usage for a given text using the Free Plan (20,000 characters/month).
#     Returns character count, percentage of monthly quota used, and estimated TTS time (in minutes).
#     """
#     num_characters = len(text)
#     monthly_quota = 20000
#     percentage_used = round((num_characters / monthly_quota) * 100, 2)

#     # Estimating time: 20,000 characters ≈ 10 minutes of high-quality TTS
#     # So, 1 minute ≈ 2000 characters
#     estimated_minutes_high_quality = round(num_characters / 2000, 2)
#     estimated_minutes_flash = round(num_characters / 1000, 2)
#     estimated_minutes_conversational = round(num_characters / 1333.33, 2)

#     return {
#         "character_count": num_characters,
#         "percent_of_free_quota_used": percentage_used,
#         "estimated_duration_minutes": {
#             "high_quality": estimated_minutes_high_quality,
#             "flash": estimated_minutes_flash,
#             "conversational_ai": estimated_minutes_conversational
#         },
#         "note": "Estimates are based on Eleven Labs' Free Plan: 20,000 characters per month."
#     }
from langchain_core.tools import tool

@tool
def token_calculate(text: str) -> dict:
    """
    Calculates character count, estimated duration, percentage of monthly quota used,
    and estimated Eleven Labs credits to be deducted from the free 10,000 monthly credits.
    """

    num_characters = len(text)
    monthly_quota_chars = 20000
    total_credits = 10000  # Free plan credits
    chars_per_credit = 2   # Based on 1 credit ≈ 2 characters

    # Calculate
    percent_quota_used = round((num_characters / monthly_quota_chars) * 100, 2)
    estimated_credits = round(num_characters / chars_per_credit, 2)

    # Estimated minutes
    estimated_minutes_high_quality = round(num_characters / 2000, 2)
    estimated_minutes_flash = round(num_characters / 1000, 2)
    estimated_minutes_conversational = round(num_characters / 1333.33, 2)

    percent_credits_used = round((estimated_credits / total_credits) * 100, 2)

    return {
        "character_count": num_characters,
        "estimated_credits_deducted": estimated_credits,
        "percent_of_free_quota_used": percent_quota_used,
        "percent_of_credits_used": percent_credits_used,
        "estimated_duration_minutes": {
            "high_quality": estimated_minutes_high_quality,
            "flash": estimated_minutes_flash,
            "conversational_ai": estimated_minutes_conversational
        },
        "note": (
            "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. "
            "1 credit ≈ 2 characters."
        )
    }


# Agent Eleven Labs

In [ ]:

# # eleven_labs_assistant = create_react_agent(
# #     model=model,
# #     tools=[token_calculate],
# #     prompt="""
# # You are an assistant helping users estimate how much it will cost to convert a given text to speech using Eleven Labs.

# # Your task:
# # - Analyze the input text provided to you.
# # - Use the `token_calculate` tool to compute the number of characters in the text and the estimated cost in USD.
# # - Return the result in the following JSON format:

# # {
# #   "character_count": <number of characters>,
# #   "estimated_cost_usd": <approximate cost in USD>
# # }

# # Be precise and always use the `token_calculate` tool to generate your response.
# # """,
# #     name="eleven_labs_assistant"
# # )
# eleven_labs_assistant = create_react_agent(
#     model=model,
#     tools=[token_calculate],
#     prompt="""
# You are an assistant that helps users estimate how much of their Eleven Labs Free Plan quota will be used when converting a given text to speech.

# Instructions:
# - Take the user's input text.
# - Use the `token_calculate` tool to compute:
#   - Total character count.
#   - What percent of the 20,000 monthly characters it uses.
#   - Estimated duration (in minutes) if converted using:
#     - High-quality TTS (Multilingual v2),
#     - Flash (v2.5),
#     - Conversational AI.

# Respond in this JSON format:

# {
#   "character_count": ...,
#   "percent_of_free_quota_used": ...,
#   "estimated_duration_minutes": {
#     "high_quality": ...,
#     "flash": ...,
#     "conversational_ai": ...
#   },
#   "note": "..."
# }

# Always call the `token_calculate` tool. Do not guess.
# """,
#     name="eleven_labs_assistant"
# )
eleven_labs_assistant = create_react_agent(
    model=model,
    tools=[token_calculate],
    prompt="""
You are an assistant that helps users estimate how many credits will be consumed from their Eleven Labs Free Plan when converting a given text to speech.

Instructions:
- Analyze the input text.
- Use the `token_calculate` tool.
- Return results in this JSON format:

{
  "character_count": ...,
  "estimated_credits_deducted": ...,
  "percent_of_credits_used": ...,
  "percent_of_free_quota_used": ...,
  "estimated_duration_minutes": {
    "high_quality": ...,
    "flash": ...,
    "conversational_ai": ...
  },
  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."
}

Always call the `token_calculate` tool and do not guess.
""",
    name="eleven_labs_assistant"
)


In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='57191bf6-7f1c-4fbf-9db8-be3460f12c28'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0uHyFR32sVgg8WA21WuQYv0i', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_credits_deducted": 197.5,\n  "percent_of_credits_used": 1.98,\n  "percent_of_free_quota_used": 1.98,\n  "estimated_duration_minutes": {\n    "high_quality": 0.2,\n    "flash": 0.4,\n    "conversational_ai": 0.3\n  },\n  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."\n}'

# 11 Agent Response

In [ ]:
text="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each breath. Let your thoughts come and go like clouds in the sky. No need to follow them or push them away. Just return to the rhythm of your breath—anchoring yourself in the present moment. You are safe. You are calm. You are grounded. Inhale deeply through your nose… and exhale gently through your mouth. With each breath, feel a sense of peace expanding within you. Now, bring to mind a feeling of gratitude. It could be for a person, a place, or even just the gift of this moment. Let that gratitude warm your heart. Take one more deep breath in… and sigh it out. As we near the end of this meditation, slowly bring your awareness back to your surroundings. Wiggle your fingers and toes. Gently roll your shoulders. When you're ready, open your eyes. Take a moment to notice how you feel. Carry this calm and clarity with you into the rest of your day. Thank you for taking this time for yourself."

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=text)
    ]
})
eleven_response["messages"]

[HumanMessage(content="Welcome to this 10-minute guided meditation. Begin by finding a comfortable seated or lying position. Gently close your eyes. Let your hands rest softly on your lap or by your sides. Take a deep breath in… and exhale slowly. Feel the weight of your body being supported by the ground beneath you. Now, take another deep breath in… and let it go. With each breath, allow your body to relax a little more. Soften your shoulders. Unclench your jaw. Relax the muscles around your eyes. Let go of any tension you may be holding. Bring your attention to your breath. Simply notice the inhale… and the exhale. Don’t try to change it. Just observe it as it flows naturally in and out. If your mind starts to wander, that’s okay. Gently bring it back to your breath. Now, take a moment to scan your body from head to toe. Notice any areas of tightness, and invite those areas to soften. Relax your forehead, your neck, your chest, your belly. Let everything melt and loosen with each br

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 1873,\n  "estimated_credits_deducted": 936.5,\n  "percent_of_credits_used": 9.37,\n  "percent_of_free_quota_used": 9.37,\n  "estimated_duration_minutes": {\n    "high_quality": 0.94,\n    "flash": 1.87,\n    "conversational_ai": 1.4\n  },\n  "note": "Based on Eleven Labs Free Plan: 20,000 characters/month ≈ 10,000 credits. 1 credit ≈ 2 characters."\n}'

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='ab1e380c-8572-4ba8-82af-c40d0fcd99c2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_EtVHdjvkZP9RC5Iud7xoRxk0', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "percent_of_free_quota_used": 1.98,\n  "estimated_duration_minutes": {\n    "high_quality": 0.2,\n    "flash": 0.4,\n    "conversational_ai": 0.3\n  },\n  "note": "Estimates are based on Eleven Labs\' Free Plan: 20,000 characters per month."\n}'

In [ ]:
eleven_response = eleven_labs_assistant.invoke({
    "messages": [
        HumanMessage(content=response["messages"][-1].content)
    ]
})
eleven_response["messages"]

[HumanMessage(content='{\n  "answer": "The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat. Its meaning is: \'We meditate on the transcendental glory of the Deity Supreme, Who is inside the heart of the earth, inside the life of the sky, and inside the soul of the Heaven. May He stimulate and illumine our minds.\'",\n  "source_documents": [\n    {\n      "title": "Bhagavad Gita",\n      "page": "Assumed context",\n      "snippet": "Aum bhur bhuvah svah Tat savitur varenyam Bhargo devasya dhimahi Dhiyo yo nah prachodayat."\n    }\n  ]\n}', additional_kwargs={}, response_metadata={}, id='d09e2e6c-8577-462b-bcb7-0067097a5d82'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1XYXkzyBoGZdDvchWU3lyEkz', 'function': {'arguments': '{"text":"The Gayatri mantra is a sacred mantra from the Bhagavad Gita, which is as follows: Aum bhur bhuvah svah Tat savit

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_cost_usd": 0.0395\n}'

In [ ]:
eleven_response["messages"][-1].content

'{\n  "character_count": 395,\n  "estimated_cost_usd": 0.0395\n}'